<h1><center> Social Network Data Web-Scraping: VK API </center></h1>
<h1><center>ANR-Lab, 14th Summer School</center></h1>
<h2><center>Lika Kapustina, Research Assistant</center></h2>
<h3><center>lvkapustina@edu.hse.ru</center></h3>

Right now we are at a workshop run by ANR-Lab, and the other Summer School workshops are predominantly about network analysis. **But is any kind of data analysis possible without...the data itself?**

My own professional interests are related primarily to data collection and preprocessing; **having web-scraping and data preparation skills allows you to formulate and test a much wider range of scientific and business hypotheses** without suffering from the hard questions "where do I even get the data?" and without spending personal money or company budget on access to companies API.

Today we're going to talk about **how you can use Python skills to retrieve data from the Russian social network Vkontakte** via its API.

## "VKontakte" and its API

What is VKontakte? VKontakte is one of the most popular social networks in the former Soviet Union. It was created in 2006 under the leadership of Pavel Durov (also known as the creator of the messenger Telegram); as of May 2023, the average monthly audience of VKontakte was 81.5 million users.

VKontakte has a handy functionality - you can communicate with friends, add people to your friends list, subscribe to communities (and much more). For purposes of the social network analysis you can find and use information on:

* News;
* Friends;
* Your(!) messages;
* Communities, its posts and subscribers, and so on..

If because of some reasons you have never had experience with VK, here is an example of VK personal page and some interface.
![screen_1.png](screen_1.png)

In addition to this large audience size and the data that resides in the social networking space, VKontakte has a fairly simple and easy-to-use API, even for novice researchers, through which you can quickly get open data from the social network. 

What is an API?
* **API** (if we try to formulate it short and quickly) - it is a set of methods, provided by the source site itself (social media, for example), through which we can retrieve data from the service with specific requests. For example: Twitter has its own API, HH.ru has an API, and so russian social media "Vkontakte" has its own API which is called "VK API".

* **What data can you retrieve?** Your page data, community posts, data about your own correspondence, information about the subscribers of a particular community, etc..

* **What kind of data can't be accessed?** Data from closed accounts (which are not your friends), data from closed communities, private information from profiles and communities.

VK API is not only used to collect data, but also to run applications: for example, you can write a python script that will "Like" all your friends' posts; or write a python script that will ban members of the community you have admin rights for. But today we won't talk about such options, we'll just talk about data collection.

So, we are gonna do a few steps:

* **Step 1**: Get a token for VK API;

* **Step 2**: Research our possibilities with VK API documentation;

* **Step 3**: Get data on communities and persons;

## Step 1: Get a token for VK API

Every big step has its own small steps:

1. **Open https://vk.com/apps?act=manage page:**

![screen_2.png](screen_2.png)

2. **Create *Standalone* app:**

![screen_3.png](screen_3.png)

3. **Click on "Manage" and look to the page link:**
>For example, I got `https://vk.com/editapp?id=666` link, so `666` is my future client_id

4. **Click on Settings and click and change `App Status`:**
![screen_3.5.png](screen_3.5.png)

5. **Create your own link:**
> `https://oauth.vk.com/authorize?client_id=1&display=page&scope=friends&response_type=token&v=5.92`

Let's look on this one. You need to change some arguments:

* `client_id` = should be your client_id that you got in the previous step;
* `v` = should be the last version of VK API (for july 2023, you can use `5.131` or `5.92`);
* `scope` = should be the types of data that you want to get access with your token (for example `friends,groups`), **but** I always use `scope=266242` and this works (allows us to get data on friends, groups and your messages).

So, if I want to use 5.131 version of VK API and get data on friends, groups and messages, I need to use link:

> `https://oauth.vk.com/authorize?client_id=1&display=page&scope=266242&response_type=token&v=5.131`

6. **Open this link and copy access_token that starts from `vk1` substring:** 

![screen_3.5.5](screen_3.5.5.png)
> `https://oauth.vk.com/blank.html#access_token=vk1.a.f1111ckm-ueqh9psP5TaJOwH87_tfwPULIYi1ADCopA9rwrzO20VFkEC6MlP25czXaatmy9p2I4T6pOhf-K8HVT4FX7Ii_Jcc4l9ve4-gBivUt5odnDTiaMmyUWM-gzF9sX1mXX83a1-sl75_mkNK0drewOjoMRRDfoToWiqOJuIT53rf-uh2zid6cn0f1FosKDMMM1fK9H1KHiSJPDK1XRIw&expires_in=86400&user_id=666`

* `access_token=` - is your access token (including both `expires_in` and `user_id`);
* `expires_in=` - is the number of seconds after which this particular `access_token` expires;
* `user_id=666` - is your user id;

<font color='red'><b>Important! Don't give your link anybody. A person with bad intentions and technical skills will be able to do different things on behalf of your account</b></font>

In [1]:
# so, let's save your data:

# 1. token

with open('token.txt', 'w') as file:
    file.write('YOUR TOKEN HERE')
    
# 2. id of your profile

with open('id.txt', 'w') as file:
    file.write('YOUR VK ID HERE')

## Step 2: Let's research our possibilities with VK API documentation;

VK API offers developers and researchers not only a user-friendly API, but also detailed documentation - an entire encyclopaedia listing all the features and possible parameters!

For example, in our next step we will get data on persons by their ids. How to understand which method and how we need to use?

1. **Open "Documentation" page by this [link](https://dev.vk.com/method)**

![screen_4.png](screen_4.png)

2. **Choose needed subpage in the left menu and click on it. In our case, its [Users](https://dev.vk.com/method/users):**

![screen_5.png](screen_5.png)

3. **Choose needed method and click on it. For our purposes, it will be [Users.get](https://dev.vk.com/method/users.get):**

![screen_6.png](screen_6.png)

What's next? We can explore our parameters and use it in our next step for web-scraping. 

<h3><b>Task for your practice: imagine, that you need to find documentation on method that will be search podcasts in VKontakte. Find it in documentation and print its name lower</b></h3>

<h3><font color='blue'><i>Your answer:</i> __________________  <font color='blue'></h3>

# Step 3: Get data on persons

So, for today we will learn how to work with VKontakte personal profiles, preprocess this data and how to handle with most common problems and errors. Step-by-step, you will know how:

1. Get data on one person;
2. Get data on a few persons at once and preprocess it to `pandas.DataFrame`;
3. Handle most common problems with VK API methods (overall).

In [2]:
# Before the start, we need to import some libraries for future data scraping and preparation
# use !pip install packagename if you see the ModuleNotFoundError
import requests
import numpy as np
from datetime import datetime
from time import sleep
import pandas as pd
import tqdm
import re
import math
import warnings
warnings.filterwarnings("ignore")

In [3]:
# token download, id and version refinement
with open('id.txt') as file:
    myid = file.read()

# version of API 
version = '5.131'

# load the token from a file on the computer
with open('token.txt') as file:
    token = file.read()

## Step 3.1: Get data on one user:

What we are gonna do now? Every user in VKontakte has its own id. Let's try to write code that will collect information on user by his/her id.

For this purpose, we need to use one of VK API method - `users.get`. We can find documentation on it by [link](https://dev.vk.com/method/users.get). Let's again look on it!

![screen_7.png](screen_7.png)

**What is the best algorithm for work with VK API? You should:**

1. Find needed method in VK API documentation;
2. Check possible parameters;
3. Decide which parameters you want to use;
4. Gather all of your parameters and your own data (token) in one request;
5. Send it and prepare the data for future analysis.

**So, in case with users** - what type of data we want to collect? Let's choose these parameters:
* `sex` - users sex;
* `about` - small description that user write by him/herself;
* `screen_name` - screen_name;
* `bdate` - birthday date;
* `city` - city of user;
* `country` - country of user;
* `education` - some basic information on education of user;
* `universities` - some basic information on university of user;
* `friend_status` - whether (`1`) or not (`0`) the user is our friend.

Meaning of this values you can find by this [link](https://dev.vk.com/reference/objects/user)

**What all of our responses will contain?**

For example, if you want to get data on user with `id = '53083705'` and parameters which was specified before, link for getting data on user by his/her will contain this parts:

```Python
parameters = 'fields=sex,about,screen_name,bdate,city,country,education,universities,friend_status'
method = 'users.get'
version = '5.131'
access_token = (access token that you created before)
user_ids = '53083705'
```

And will look like this:

```Python
f'https://api.vk.com/method/{method}?{parameters}&v={version}&access_token={token}&user_ids={user_ids}'
```

**How to get user id?** Here you have two possibilities: 

1. **Easy way:** use services such as ["Узнать ID страницы или группы"](https://regvk.com/id/) to get id of specific user;
2. **Hard but nice way**: use method `utils.resolveScreenName`.

<h3><b>Task with star*: write code that will get user's id by it's screen name (address of the personal page)</b></h3>

For example, my personal link is `https://vk.com/lika.kapustina` which means that `lika.kapustina` is my screen name; but for user with personal link `https://vk.com/id136161093` screen name will be `id136161093`. Use `if-else` statements and method `utils.resolveSreenName` to write a code that will get user id by link on his/her personal page:

In [12]:
# YOUR CODE HERE

**But let's go back to the code!**

In [4]:
# specify all of the parameters
parameters = 'fields=sex,about,screen_name,bdate,city,country,education,universities,friend_status'
method = 'users.get'
version = '5.131'
user_ids = '53083705'
url = f'https://api.vk.com/method/{method}?{parameters}&v={version}&access_token={token}&user_ids={user_ids}'
response = requests.get(url)
users_data = response.json()

In [5]:
users_data # data on Dmitriy Medvedev, ex-Russian President

{'response': [{'id': 53083705,
   'bdate': '14.9.1965',
   'city': {'id': 1, 'title': 'Moscow'},
   'country': {'id': 1, 'title': 'Russia'},
   'about': '',
   'university': 1,
   'university_name': 'СПбГУ',
   'faculty': 19,
   'faculty_name': 'Юридический факультет',
   'graduation': 1987,
   'education_form': 'Full-time',
   'education_status': 'Candidate of Sciences',
   'universities': [{'chair': 270,
     'chair_name': 'Гражданского права\r\n',
     'city': 2,
     'country': 1,
     'education_form': 'Full-time',
     'education_form_id': 1,
     'education_status': 'Candidate of Sciences',
     'education_status_id': 9,
     'faculty': 19,
     'faculty_name': 'Юридический факультет',
     'graduation': 1987,
     'id': 1,
     'name': 'СПбГУ'}],
   'sex': 2,
   'screen_name': 'dm',
   'friend_status': 0,
   'first_name': 'Dmitry',
   'last_name': 'Medvedev',
   'can_access_closed': True,
   'is_closed': False}]}

In [7]:
# what is users_data object?
type(users_data) # dict, because we preprocessed it as json before

dict

In [6]:
# how to work with data in it?
users_data['response'] # main information

[{'id': 53083705,
  'bdate': '14.9.1965',
  'city': {'id': 1, 'title': 'Moscow'},
  'country': {'id': 1, 'title': 'Russia'},
  'about': '',
  'university': 1,
  'university_name': 'СПбГУ',
  'faculty': 19,
  'faculty_name': 'Юридический факультет',
  'graduation': 1987,
  'education_form': 'Full-time',
  'education_status': 'Candidate of Sciences',
  'universities': [{'chair': 270,
    'chair_name': 'Гражданского права\r\n',
    'city': 2,
    'country': 1,
    'education_form': 'Full-time',
    'education_form_id': 1,
    'education_status': 'Candidate of Sciences',
    'education_status_id': 9,
    'faculty': 19,
    'faculty_name': 'Юридический факультет',
    'graduation': 1987,
    'id': 1,
    'name': 'СПбГУ'}],
  'sex': 2,
  'screen_name': 'dm',
  'friend_status': 0,
  'first_name': 'Dmitry',
  'last_name': 'Medvedev',
  'can_access_closed': True,
  'is_closed': False}]

In [8]:
# it's work like just regular python dictionaries
my_dict = {1: 'one', 2: 'two', 3:'three'}
my_dict[3]

'three'

In [17]:
users_data['response'][0] # data on first user; in our case - on first and last.

{'id': 53083705,
 'bdate': '14.9.1965',
 'city': {'id': 1, 'title': 'Moscow'},
 'country': {'id': 1, 'title': 'Russia'},
 'about': '',
 'university': 1,
 'university_name': 'СПбГУ',
 'faculty': 19,
 'faculty_name': 'Юридический факультет',
 'graduation': 1987,
 'education_form': 'Full-time',
 'education_status': 'Candidate of Sciences',
 'universities': [{'chair': 270,
   'chair_name': 'Гражданского права\r\n',
   'city': 2,
   'country': 1,
   'education_form': 'Full-time',
   'education_form_id': 1,
   'education_status': 'Candidate of Sciences',
   'education_status_id': 9,
   'faculty': 19,
   'faculty_name': 'Юридический факультет',
   'graduation': 1987,
   'id': 1,
   'name': 'СПбГУ'}],
 'sex': 2,
 'screen_name': 'dm',
 'friend_status': 0,
 'first_name': 'Dmitry',
 'last_name': 'Medvedev',
 'can_access_closed': True,
 'is_closed': False}

In [21]:
# But how to get the value of the attribute?
print(users_data['response'][0]['first_name']) # first name
print(users_data['response'][0]['last_name']) # last_name
print(users_data['response'][0]['bdate']) # date of birthday

Dmitry
Medvedev
14.9.1965


<h3>Task for you: print name of country where Dmitry Medvedev lives and name of university which he graduated</h3>

In [ ]:
# YOUR CODE HERE - NAME OF COUNTRY OF LIVING

In [ ]:
# YOUR CODE HERE - NAME OF UNIVERSITY

## 3.2 Getting data on a few persons by one request:

Let's remember on seminar of which Laboratory we are right now.. Right, if in future we want to work with data for future network analysis, we need to get data on at least few people. How to do it?

For this, we need to specify the last part of our request, `user_ids`

We need to write a few ids in one python string. For example, `user_ids=53083705,453382669,12086666'

In [28]:
parameters = 'fields=sex,about,screen_name,bdate,city,country,education,universities,friend_status'
method = 'users.get'
version = '5.131'
user_ids = '53083705,453382669,12086666'
url = f'https://api.vk.com/method/{method}?{parameters}&v={version}&access_token={token}&user_ids={user_ids}'
response = requests.get(url)
users_data = response.json()

In [29]:
# But how to deal with this structure and create pandas.DataFrame based on this?
users_data

{'response': [{'id': 53083705,
   'bdate': '14.9.1965',
   'city': {'id': 1, 'title': 'Moscow'},
   'country': {'id': 1, 'title': 'Russia'},
   'about': '',
   'university': 1,
   'university_name': 'СПбГУ',
   'faculty': 19,
   'faculty_name': 'Юридический факультет',
   'graduation': 1987,
   'education_form': 'Full-time',
   'education_status': 'Candidate of Sciences',
   'universities': [{'chair': 270,
     'chair_name': 'Гражданского права\r\n',
     'city': 2,
     'country': 1,
     'education_form': 'Full-time',
     'education_form_id': 1,
     'education_status': 'Candidate of Sciences',
     'education_status_id': 9,
     'faculty': 19,
     'faculty_name': 'Юридический факультет',
     'graduation': 1987,
     'id': 1,
     'name': 'СПбГУ'}],
   'sex': 2,
   'screen_name': 'dm',
   'friend_status': 0,
   'first_name': 'Dmitry',
   'last_name': 'Medvedev',
   'can_access_closed': True,
   'is_closed': False},
  {'id': 453382669,
   'bdate': '5.11.1981',
   'city': {'id': 1, 

In [32]:
# Short answer - we need to use loop for. For! example:
number_of_object = len(users_data['response'])
print(f'We have data on {number_of_object} persons in object "users_data"')

We have data on 3 persons in object "users_data"


In [33]:
# If we use loop for, we can print needed object from response using its index.
for i in range(3):
    print(users_data['response'][i])

{'id': 53083705, 'bdate': '14.9.1965', 'city': {'id': 1, 'title': 'Moscow'}, 'country': {'id': 1, 'title': 'Russia'}, 'about': '', 'university': 1, 'university_name': 'СПбГУ', 'faculty': 19, 'faculty_name': 'Юридический факультет', 'graduation': 1987, 'education_form': 'Full-time', 'education_status': 'Candidate of Sciences', 'universities': [{'chair': 270, 'chair_name': 'Гражданского права\r\n', 'city': 2, 'country': 1, 'education_form': 'Full-time', 'education_form_id': 1, 'education_status': 'Candidate of Sciences', 'education_status_id': 9, 'faculty': 19, 'faculty_name': 'Юридический факультет', 'graduation': 1987, 'id': 1, 'name': 'СПбГУ'}], 'sex': 2, 'screen_name': 'dm', 'friend_status': 0, 'first_name': 'Dmitry', 'last_name': 'Medvedev', 'can_access_closed': True, 'is_closed': False}
{'id': 453382669, 'bdate': '5.11.1981', 'city': {'id': 1, 'title': 'Moscow'}, 'country': {'id': 1, 'title': 'Russia'}, 'about': '', 'university': 236, 'university_name': 'МГИМО МИД России', 'faculty

In [34]:
# Let's get information on first and last names of each other
for i in range(3):
    print(users_data['response'][i]['first_name'])
    print(users_data['response'][i]['last_name'])
    print('\n') # Nice!

Dmitry
Medvedev


Ksenia
Sobchak


Sergey
Mironov




In [43]:
# So, how we use for loop?
df = pd.DataFrame() # empty pandas.DataFrame

for i in range(3):
    first_name = users_data['response'][i]['first_name']
    second_name = users_data['response'][i]['last_name']
    birthday = users_data['response'][i]['bdate']
    education = users_data['response'][i]['university_name']
    one_politician = pd.DataFrame([first_name, second_name, birthday, education]).transpose()
    one_politician.columns = ['first_name', 'last_name', 'birthday', 'university']
    df = pd.concat([df, one_politician])
    
df # here are

,first_name,last_name,birthday,university
0,Dmitry,Medvedev,14.9.1965,СПбГУ
0,Ksenia,Sobchak,5.11.1981,МГИМО МИД России
0,Sergey,Mironov,14.2.1953,Горный университет (бывш. СПГГУ)


In [42]:
# And let's try to get other information on russian politicians
df = pd.DataFrame() # empty pandas.DataFrame

for i in range(3):
    first_name = users_data['response'][i]['first_name']
    second_name = users_data['response'][i]['last_name']
    birthday = users_data['response'][i]['bdate']
    education_university = users_data['response'][i]['university_name']
    
    # Try to get faculty name
    try:
        education_faculty = users_data['response'][i]['universities'][0]['faculty_name']
        
    # If face error, assign value numpy.nan to value education_faculty
    except:
        education_faculty = np.nan
    
    one_politician = pd.DataFrame([first_name, second_name, birthday, education_university, education_faculty]).transpose()
    one_politician.columns = ['first_name', 'last_name', 'birthday', 'university', 'faculty']
    df = pd.concat([df, one_politician])
    
df # here are

,first_name,last_name,birthday,university,faculty
0,Dmitry,Medvedev,14.9.1965,СПбГУ,Юридический факультет
0,Ksenia,Sobchak,5.11.1981,МГИМО МИД России,NaN
0,Sergey,Mironov,14.2.1953,Горный университет (бывш. СПГГУ),NaN


In [44]:
# If we won't use try-except statement, we will face problems...
df = pd.DataFrame() # empty pandas.DataFrame

for i in range(3):
    first_name = users_data['response'][i]['first_name']
    second_name = users_data['response'][i]['last_name']
    birthday = users_data['response'][i]['bdate']
    education_university = users_data['response'][i]['university_name']
    
    education_faculty = users_data['response'][i]['universities'][0]['faculty_name']
    
    one_politician = pd.DataFrame([first_name, second_name, birthday, education_university, education_faculty]).transpose()
    one_politician.columns = ['first_name', 'last_name', 'birthday', 'university', 'faculty']
    df = pd.concat([df, one_politician])
    
df # here are

KeyError: 'faculty_name'

<h3>Task for you: based on code before, get `pandas.DataFrame` with `screen name` and year of graduation from the first university (with index 0)</h3>

In [ ]:
# YOUR CODE HERE

## 3.3 What is an important? Handling errors and problems:

1. **Don't change your Wi-fi address during web-scraping process and update your token not less than one in a 24 hours**: either it will lead to `error_code 5`

In [11]:
parameters = 'fields=sex,about,screen_name,bdate,city,country,education,universities,friend_status'
method = 'users.get'
version = '5.131'
user_ids = '53083705'
url = f'https://api.vk.com/method/{method}?{parameters}&v={version}&access_token={token}&user_ids={user_ids}'
response = requests.get(url)
users_data = response.json()
users_data

{'error': {'error_code': 5,
  'error_msg': 'User authorization failed: access_token was given to another ip address',
  'error_text': 'Failed to connect to\xa0network. Please close the\xa0page and\xa0try again.',
  'request_params': [{'key': 'fields',
    'value': 'sex,about,screen_name,bdate,city,country,education,universities,friend_status'},
   {'key': 'v', 'value': '5.131'},
   {'key': 'expires_in', 'value': '86400'},
   {'key': 'user_id', 'value': '441721976'},
   {'key': 'user_ids', 'value': '53083705'},
   {'key': 'method', 'value': 'users.get'},
   {'key': 'oauth', 'value': '1'}],
  'view': 'alert'}}

2. **Check correct name of parameters:** `users_ids` is not equal to `user_i`, which leads to specific error - you will get information on yourself..

In [23]:
parameters = 'fields=sex,about,screen_name,bdate,city,country,education,universities,friend_status'
method = 'users.get'
version = '5.131'
user_ids = '53083705'
url = f'https://api.vk.com/method/{method}?{parameters}&v={version}&access_token={token}&user_i={user_ids}'
response = requests.get(url)
users_data = response.json()
users_data

{'response': [{'id': 441721976,
   'bdate': '24.9.1917',
   'city': {'id': 1, 'title': 'Moscow'},
   'country': {'id': 1, 'title': 'Russia'},
   'about': '',
   'university': 128,
   'university_name': 'НИУ ВШЭ (ГУ-ВШЭ)',
   'faculty': 0,
   'faculty_name': '',
   'graduation': 0,
   'universities': [{'city': 1,
     'country': 1,
     'id': 128,
     'name': 'НИУ ВШЭ (ГУ-ВШЭ)'}],
   'sex': 2,
   'screen_name': 'lika.kapustina',
   'friend_status': 0,
   'first_name': 'Lika',
   'last_name': 'Kapustina',
   'can_access_closed': True,
   'is_closed': True}]}

3. **Check whether you have access to data or not:** using VK API and possible all of the legal methods of web-scraping, you can't access and scrape private data: data on closed pages or communities.

So, try to get my information - if you aren't my friend, VK API won't let you do this :)

In [ ]:
parameters = 'fields=sex,about,screen_name,bdate,city,country,education,universities,friend_status'
method = 'users.get'
version = '5.131'
user_ids = '441721976'
url = f'https://api.vk.com/method/{method}?{parameters}&v={version}&access_token={token}&user_id={user_ids}'
response = requests.get(url)
users_data = response.json()
users_data # You can check it by yourself :)

In [ ]:
# CHECK ME...

## 3.4 Some optional homework:

Well, when I finished part of work with `users` with VK API, I understood that probably workshop with other parts (work with `groups` and `posts`) of VK API methods would take way much more time and would have kept me with you for too long. 

**But I can offer you next** – I understand that programmming and data analysis is nothing without practice, so try to use method `wall.get` and create function `get_last_100_posts(group_id)`, that will get id of `group` and return `pandas.DataFrame` with last 100 posts of this group.

**It should work like this:**

In [27]:
get_100_last_posts_from_one_group('139461971')

,date,group_name,group_members,post_local_id,from_id,text,comments,likes,reposts,views
0,2023-07-03 10:40:00,Прикладной сетевой анализ НИУ ВШЭ,294,617,-139461971,ANR-Lab провела коммерческое исследование для ...,0,4,1,65
1,2023-06-28 09:44:00,Прикладной сетевой анализ НИУ ВШЭ,294,615,-139461971,Итоги защиты ВКР выпускников MASNA-2023 16-17...,0,4,0,200
2,2023-06-27 10:59:00,Прикладной сетевой анализ НИУ ВШЭ,294,613,-139461971,"«У нас получается создать коллектив, на которы...",0,10,0,134
3,2023-06-26 09:53:12,Прикладной сетевой анализ НИУ ВШЭ,294,611,-139461971,Интервью Ивана Климова каналу Nasledie.digital...,0,4,0,78
4,2023-06-24 18:13:36,Прикладной сетевой анализ НИУ ВШЭ,294,610,-139461971,26 июня в 15:00 состоится семинар Международно...,0,2,1,139
...,...,...,...,...,...,...,...,...,...,...
95,2021-08-05 19:16:00,Прикладной сетевой анализ НИУ ВШЭ,294,506,-139461971,Лучшие показатели успеха программы: выпускники...,0,14,0,403
96,2021-07-31 13:12:00,Прикладной сетевой анализ НИУ ВШЭ,294,504,-139461971,Управляйте отношениями (между данными): обзор ...,0,19,0,219
97,2021-07-26 18:41:54,Прикладной сетевой анализ НИУ ВШЭ,294,502,-139461971,Сердце и разум: анализ настроений в R Как уз...,0,16,1,225
98,2021-07-16 19:43:01,Прикладной сетевой анализ НИУ ВШЭ,294,501,-139461971,Открывая возможности темпорального сетевого ан...,0,21,0,227


In [ ]:
# YOUR CODE HERE

Either way, whether you succeed in writing the function or stop at some part of the code, **send me your code in .ipynb format** at *lkapustina@hse.ru* and I will try to give you feedback :)

Remember - methods and skills of web-scraping is very useful (mostly because very small percentage of researchers are familiar with them), and check on updates of **Applied Network Research Laboratory** and **Master of Applied Social Network Analysis program** :) for future interesting news, events and education calls

<center><h2><font color='pink'>Thank you for an attention!</font></h2></center>